DJAGHLOUL AYOUB AHMED AMINE

1. Environment Setup
Make sure you have the necessary libraries installed. For neural networks, libraries like TensorFlow and Keras are widely used. You can install them via pip if you haven't already:

In [5]:
pip install tensorflow keras


2. Data Preprocessing
Your data preprocessing steps will be similar to what you did for SVM. However, for neural networks, you might want to tokenize the text data and convert it into sequences of integers. Keras provides utilities for this:

In [3]:
import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.metrics import classification_report

# Fonction de prétraitement du texte
def preprocess_text(text):
    text = text.lower()  # Passage en minuscules
    text = re.sub(r'\W', ' ', text)  # Suppression des caractères non alphanumériques
    text = re.sub(r'\s+', ' ', text)  # Réduction des espaces multiples
    return text

# Fonction pour charger les données
def charge_donnees(train_file, dev_file, test_file):
    train_data = pd.read_csv(train_file, sep='\t', names=['tweet_id', 'sentiment', 'text'])
    dev_data = pd.read_csv(dev_file, sep='\t', names=['tweet_id', 'sentiment', 'text'])
    test_data = pd.read_csv(test_file, sep='\t', names=['tweet_id', 'sentiment', 'text'])
    return train_data, dev_data, test_data

# Prétraitement et préparation des données
def prepare_data(data, tokenizer):
    sequences = tokenizer.texts_to_sequences(data['text'].apply(preprocess_text))
    padded_sequences = pad_sequences(sequences, maxlen=100)
    return padded_sequences




# Construction du modèle avec des ajustements
def build_model(vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length=100))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.6))  # Ajustement du taux de dropout
    model.add(BatchNormalization())
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.6))  # Ajustement du taux de dropout
    model.add(BatchNormalization())
    model.add(LSTM(32))  # Ajout d'une couche LSTM supplémentaire
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Chemins vers les fichiers de données
train_file = 'twitter-2013train-A.txt'
dev_file = 'twitter-2013dev-A.txt'
test_file = 'twitter-2013test-A.txt'


# Charger les données
train_data, dev_data, test_data = charge_donnees(train_file, dev_file, test_file)


# Créer un tokeniseur pour tout le vocabulaire
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'].apply(preprocess_text))

# Préparation des données
X_train = prepare_data(train_data, tokenizer)
X_dev = prepare_data(dev_data, tokenizer)
X_test = prepare_data(test_data, tokenizer)

# Taille du vocabulaire
vocab_size = len(tokenizer.word_index) + 1  # Ajouter 1 pour le mot de padding

# Encodage des étiquettes
label_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}
y_train = to_categorical([label_mapping[sentiment] for sentiment in train_data['sentiment']], num_classes=3)
y_dev = to_categorical([label_mapping[sentiment] for sentiment in dev_data['sentiment']], num_classes=3)
y_test = to_categorical([label_mapping[sentiment] for sentiment in test_data['sentiment']], num_classes=3)

# Construction du modèle
model = build_model(vocab_size)


# Entraînement du modèle avec Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)  # Ajustement de la patience
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_dev, y_dev), callbacks=[early_stopping])  # Ajustement du nombre d'époques

# Évaluation du modèle données de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision du modèle Test Set: {accuracy * 100:.2f}%")
# Prédire les étiquettes pour l'ensemble de test
y_pred_test = model.predict(X_test)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
y_true_test_classes = np.argmax(y_test, axis=1)

# Calculer et afficher le rapport de classification
print("Classification Report for the Neural Network Model on Test Set:")
print(classification_report(y_true_test_classes, y_pred_test_classes))

# Évaluation du modèle données de dev
loss, accuracy = model.evaluate(X_dev, y_dev)
print(f"Précision du modèle Dev Set: {accuracy * 100:.2f}%")
# Prédire les étiquettes pour l'ensemble de test
y_pred_dev = model.predict(X_dev)
y_pred_dev_classes = np.argmax(y_pred_dev, axis=1)
y_true_dev_classes = np.argmax(y_dev, axis=1)

# Calculer et afficher le rapport de classification
print("Classification Report for the Neural Network Model on Dev Set:")
print(classification_report(y_true_dev_classes, y_pred_dev_classes))

Epoch 1/15
303/303 [==============================] - 47s 135ms/step - loss: 0.9989 - accuracy: 0.5164 - val_loss: 0.9928 - val_accuracy: 0.5484
Epoch 2/15
303/303 [==============================] - 41s 134ms/step - loss: 0.7547 - accuracy: 0.6971 - val_loss: 0.8980 - val_accuracy: 0.6385
Epoch 3/15
303/303 [==============================] - 41s 134ms/step - loss: 0.4738 - accuracy: 0.8344 - val_loss: 1.1668 - val_accuracy: 0.6179
Epoch 4/15
111/111 [==============================] - 4s 39ms/step - loss: 1.0973 - accuracy: 0.6380
Précision du modèle Test Set: 63.80%
111/111 [==============================] - 5s 38ms/step
Classification Report for the Neural Network Model on Test Set:
              precision    recall  f1-score   support

           0       0.66      0.66      0.66      1513
           1       0.64      0.71      0.67      1475
           2       0.57      0.39      0.46       559

    accuracy                           0.64      3547
   macro avg       0.62      0.59  

#Réseau de Neurones Amélioré sur l'Ensemble de Test:
Précision globale: 64.48%
Classe 0 (Neutral): Précision = 0.61, Rappel = 0.80, Score F1 = 0.69
Classe 1 (Positive): Précision = 0.75, Rappel = 0.58, Score F1 = 0.66
Classe 2 (Negative): Précision = 0.50, Rappel = 0.39, Score F1 = 0.44
SVM sur l'Ensemble de Test:
Précision globale: 64%
Classe -1 (Negative): Précision = 0.57, Rappel = 0.37, Score F1 = 0.45
Classe 0 (Neutral): Précision = 0.61, Rappel = 0.76, Score F1 = 0.68
Classe 1 (Positive): Précision = 0.70, Rappel = 0.62, Score F1 = 0.66

Comparaison:

Précision Globale: Le modèle RN amélioré et le SVM ont des performances presque identiques en termes de précision globale, avec une légère amélioration pour le RN (64.48% contre 64%).
Performance par Classe:
Neutral: Le RN a un meilleur rappel mais une précision légèrement inférieure pour la classe Neutral par rapport au SVM.
Positive: Le RN a une meilleure précision pour la classe Positive, mais un rappel plus faible.
Negative: Pour la classe Negative, le RN et le SVM ont des performances similaires en termes de score F1, mais le RN a une précision légèrement inférieure.

Conclusion:

Les améliorations apportées au RN ont abouti à des performances comparables à celles du SVM, avec des différences mineures dans la précision et le rappel entre les classes.
Le choix entre le RN et le SVM pourrait dépendre de facteurs spécifiques à l'application, tels que l'importance relative de la précision par rapport au rappel pour chaque classe de sentiment.
En général, le RN amélioré offre un équilibre légèrement meilleur entre la précision et le rappel pour les différentes classes par rapport au SVM, bien que les différences soient modestes.